In [85]:
import requests
import json
from web3 import Web3
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px

In [93]:
class NFTCount:
    def __init__(self, api_key, contract_address):
        self.api_key = api_key
        self.contract_address = Web3.toChecksumAddress(contract_address)
        self.w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/96bebbfc76104c5e923d42fb149b60d7'))
        with open('2nd_abi.json', 'r') as f:
            self.contract_abi = json.load(f)
        self.contract = self.w3.eth.contract(address=self.contract_address, abi=self.contract_abi['result'])

    def total_nfts(self):
        """Retrieve the total supply of NFTs."""
        all_valid_nfts = self.contract.functions.totalSupply().call()
        return all_valid_nfts // 10**18

In [88]:
class PunkVPriceChart:
    def __init__(self, contract_address):
        self.w3 = Web3(Web3.HTTPProvider('https://mainnet.infura.io/v3/96bebbfc76104c5e923d42fb149b60d7'))
        with open('4th_abi.json', 'r') as f:
            self.fourth_contract_abi = json.load(f)
        self.contract_address = contract_address

    def get_logs(self):
        fourth_address = Web3.toChecksumAddress(self.contract_address)
        fourth_contract = self.w3.eth.contract(address=fourth_address,abi=self.fourth_contract_abi['result'])
        event_signature = 'Swap'
        event_filter = fourth_contract.events[event_signature].createFilter(fromBlock='earliest', toBlock='latest')
        logs = event_filter.get_all_entries()
        return logs

    def create_dataframe(self, logs):
        df = pd.DataFrame(columns=['timestamp', 'punkIn', 'punkOut', 'etherIn', 'etherOut'])
        for log in logs:
            block_number = log['blockNumber']
            block = self.w3.eth.getBlock(block_number)
            timestamp = block['timestamp']

            if log.args.amount1In != 0 and log.args.amount0Out != 0:
                df = df.append({
                    'timestamp': timestamp,
                    'punkOut': log.args.amount0Out/10**18,
                    'etherIn': log.args.amount1In/10**18
                }, ignore_index=True)
            elif log.args.amount1Out != 0 and log.args.amount0In != 0:
                df = df.append({
                    'timestamp': timestamp,
                    'punkIn': log.args.amount0In/10**18,
                    'etherOut': log.args.amount1Out/10**18
                }, ignore_index=True)

        if 'etherIn' in df.columns and 'etherOut' in df.columns:
            df['price'] = df['etherIn'] / df['punkOut']
        else:
            df['price'] = df['etherOut'] / df['punkIn']

        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
        df.set_index('timestamp', inplace=True)
        return df

    def create_chart(self, df):
        daily_data = df.resample('D').mean()
        fig = px.line(daily_data, x=daily_data.index, y='price')

        # Set chart title and axes labels
        fig.update_layout(
            title="Price of 1 PUNKV in ETH",
            xaxis_title="Date",
            yaxis_title="Price"
        )
        fig.update_traces(connectgaps=True)

        # Show the chart
        fig.show()

In [90]:
my_token = PunkVPriceChart(contract_address = '0x0463a06fBc8bF28b3F120cd1BfC59483F099d332')
logs = my_token.get_logs()
df = my_token.create_dataframe(logs)
my_token.create_chart(df)


C:\Users\PYVelev\AppData\Local\Temp\ipykernel_15784\1157714689.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\PYVelev\AppData\Local\Temp\ipykernel_15784\1157714689.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\PYVelev\AppData\Local\Temp\ipykernel_15784\1157714689.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\PYVelev\AppData\Local\Temp\ipykernel_15784\1157714689.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({
C:\Users\PYVelev\AppData\Local\Temp\ipykernel_15784\1157714689.py:30: FutureWarning: The frame.append method is 

In [94]:
my_nft = NFTCount(api_key='76GSRJKVXQSTN6PZ3R1UM54PDXV4QYHPZP', contract_address='0x269616d549d7e8eaa82dfb17028d0b212d11232a')
total_nfts = my_nft.total_nfts()
print("Total NFTs in the vault:", total_nfts)

Total NFTs in the vault: 136
